### Standard

In [ ]:
import random
import os
import torch
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm  
import os
from PIL import Image
import shutil
import time
import yaml
from pathlib import Path



In [3]:

# Define global constants for dataset directories
DATA_DIR = '/kaggle/input/byu-locating-bacterial-flagellar-motors-2025'
TRAIN_CSV = os.path.join(DATA_DIR, 'train_labels.csv')
TRAIN_DIR = os.path.join(DATA_DIR, 'train')
TEST_DIR = os.path.join(DATA_DIR, 'test')
OUTPUT_DIR = './'
MODEL_DIR = './models'

# Create output directories if they don't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)
os.makedirs(MODEL_DIR, exist_ok=True)

# Set device: Use GPU if available; otherwise, fall back to CPU
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {DEVICE}")

# Set random seeds for reproducibility
RANDOM_SEED = 42
random.seed(RANDOM_SEED)
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed(RANDOM_SEED)
    torch.backends.cudnn.deterministic = True





Using device: cuda


### Dataset Setup

In [ ]:
# Define YOLO dataset structure and parameters
data_path = "kaggle/input/byu-locating-bacterial-flagellar-motors-2025/"
train_dir = os.path.join(data_path, "train")

# Output directories for YOLO dataset (adjust as needed)
yolo_dataset_dir = "kaggle/working/yolo_dataset"
yolo_images_train = os.path.join(yolo_dataset_dir, "images", "train")
yolo_images_val = os.path.join(yolo_dataset_dir, "images", "val")
yolo_labels_train = os.path.join(yolo_dataset_dir, "labels", "train")
yolo_labels_val = os.path.join(yolo_dataset_dir, "labels", "val")

# Create necessary directories
for dir_path in [yolo_images_train, yolo_images_val, yolo_labels_train, yolo_labels_val]:
    os.makedirs(dir_path, exist_ok=True)

# Define constants for processing
TRUST = 4       # Number of slices above and below center slice (total slices = 2*TRUST + 1)
BOX_SIZE = 24   # Bounding box size (in pixels)
TRAIN_SPLIT = 0.8  # 80% training, 20% validation

# Define a helper function for image normalization using percentile-based contrast enhancement.
def normalize_slice(slice_data):
    """
    Normalize slice data using the 2nd and 98th percentiles.
    
    Args:
        slice_data (numpy.array): Input image slice.
    
    Returns:
        np.uint8: Normalized image in the range [0, 255].
    """
    p2 = np.percentile(slice_data, 2)
    p98 = np.percentile(slice_data, 98)
    clipped_data = np.clip(slice_data, p2, p98)
    normalized = 255 * (clipped_data - p2) / (p98 - p2)
    return np.uint8(normalized)

# Define the preprocessing function to extract slices, normalize, and generate YOLO annotations.
def prepare_yolo_dataset(trust=TRUST, train_split=TRAIN_SPLIT):
    """
    Extract slices containing motors and save images with corresponding YOLO annotations.
    
    Steps:
    - Load the motor labels.
    - Perform a train/validation split by tomogram.
    - For each motor, extract slices in a range (± trust parameter).
    - Normalize each slice and save it.
    - Generate YOLO format bounding box annotations with a fixed box size.
    - Create a YAML configuration file for YOLO training.
    
    Returns:
        dict: A summary containing dataset statistics and file paths.
    """
    # Load the labels CSV
    labels_df = pd.read_csv(os.path.join(data_path, "train_labels.csv"))
    
    total_motors = labels_df['Number of motors'].sum()
    print(f"Total number of motors in the dataset: {total_motors}")
    
    # Consider only tomograms with at least one motor
    tomo_df = labels_df[labels_df['Number of motors'] > 0].copy()
    unique_tomos = tomo_df['tomo_id'].unique()
    print(f"Found {len(unique_tomos)} unique tomograms with motors")
    
    # Shuffle and split tomograms into train and validation sets
    np.random.shuffle(unique_tomos)
    split_idx = int(len(unique_tomos) * train_split)
    train_tomos = unique_tomos[:split_idx]
    val_tomos = unique_tomos[split_idx:]
    print(f"Split: {len(train_tomos)} tomograms for training, {len(val_tomos)} tomograms for validation")
    
    # Helper function to process a list of tomograms
    def process_tomogram_set(tomogram_ids, images_dir, labels_dir, set_name):
        motor_counts = []
        for tomo_id in tomogram_ids:
            # Get motor annotations for the current tomogram
            tomo_motors = labels_df[labels_df['tomo_id'] == tomo_id]
            for _, motor in tomo_motors.iterrows():
                if pd.isna(motor['Motor axis 0']):
                    continue
                motor_counts.append(
                    (tomo_id, 
                     int(motor['Motor axis 0']), 
                     int(motor['Motor axis 1']), 
                     int(motor['Motor axis 2']),
                     int(motor['Array shape (axis 0)']))
                )
        
        print(f"Will process approximately {len(motor_counts) * (2 * trust + 1)} slices for {set_name}")
        processed_slices = 0
        
        # Loop over each motor annotation
        for tomo_id, z_center, y_center, x_center, z_max in tqdm(motor_counts, desc=f"Processing {set_name} motors"):
            z_min = max(0, z_center - trust)
            z_max_bound = min(z_max - 1, z_center + trust)
            for z in range(z_min, z_max_bound + 1):
                # Create the slice filename and source path
                slice_filename = f"slice_{z:04d}.jpg"
                src_path = os.path.join(train_dir, tomo_id, slice_filename)
                if not os.path.exists(src_path):
                    print(f"Warning: {src_path} does not exist, skipping.")
                    continue
                
                # Load, normalize, and save the image slice
                img = Image.open(src_path)
                img_array = np.array(img)
                normalized_img = normalize_slice(img_array)
                dest_filename = f"{tomo_id}_z{z:04d}_y{y_center:04d}_x{x_center:04d}.jpg"
                dest_path = os.path.join(images_dir, dest_filename)
                Image.fromarray(normalized_img).save(dest_path)
                
                # Prepare YOLO bounding box annotation (normalized values)
                img_width, img_height = img.size
                x_center_norm = x_center / img_width
                y_center_norm = y_center / img_height
                box_width_norm = BOX_SIZE / img_width
                box_height_norm = BOX_SIZE / img_height
                label_path = os.path.join(labels_dir, dest_filename.replace('.jpg', '.txt'))
                with open(label_path, 'w') as f:
                    f.write(f"0 {x_center_norm} {y_center_norm} {box_width_norm} {box_height_norm}\n")
                
                processed_slices += 1
        
        return processed_slices, len(motor_counts)
    
    # Process training tomograms
    train_slices, train_motors = process_tomogram_set(train_tomos, yolo_images_train, yolo_labels_train, "training")
    # Process validation tomograms
    val_slices, val_motors = process_tomogram_set(val_tomos, yolo_images_val, yolo_labels_val, "validation")
    
    # Generate YAML configuration for YOLO training
    yaml_content = {
        'path': yolo_dataset_dir,
        'train': 'images/train',
        'val': 'images/val',
        'names': {0: 'motor'}
    }
    with open(os.path.join(yolo_dataset_dir, 'dataset.yaml'), 'w') as f:
        yaml.dump(yaml_content, f, default_flow_style=False)
    
    print(f"\nProcessing Summary:")
    print(f"- Train set: {len(train_tomos)} tomograms, {train_motors} motors, {train_slices} slices")
    print(f"- Validation set: {len(val_tomos)} tomograms, {val_motors} motors, {val_slices} slices")
    print(f"- Total: {len(train_tomos) + len(val_tomos)} tomograms, {train_motors + val_motors} motors, {train_slices + val_slices} slices")
    
    return {
        "dataset_dir": yolo_dataset_dir,
        "yaml_path": os.path.join(yolo_dataset_dir, 'dataset.yaml'),
        "train_tomograms": len(train_tomos),
        "val_tomograms": len(val_tomos),
        "train_motors": train_motors,
        "val_motors": val_motors,
        "train_slices": train_slices,
        "val_slices": val_slices
    }

# Run the preprocessing
summary = prepare_yolo_dataset(TRUST)
print(f"\nPreprocessing Complete:")
print(f"- Training data: {summary['train_tomograms']} tomograms, {summary['train_motors']} motors, {summary['train_slices']} slices")
print(f"- Validation data: {summary['val_tomograms']} tomograms, {summary['val_motors']} motors, {summary['val_slices']} slices")
print(f"- Dataset directory: {summary['dataset_dir']}")
print(f"- YAML configuration: {summary['yaml_path']}")
print("\nReady for YOLO training!")

Total number of motors in the dataset: 831
Found 362 unique tomograms with motors
Split: 289 tomograms for training, 73 tomograms for validation
Will process approximately 3267 slices for training


Processing training motors:   0%|          | 0/363 [00:00<?, ?it/s]

Will process approximately 792 slices for validation


Processing validation motors:   0%|          | 0/88 [00:00<?, ?it/s]


Processing Summary:
- Train set: 289 tomograms, 363 motors, 3262 slices
- Validation set: 73 tomograms, 88 motors, 792 slices
- Total: 362 tomograms, 451 motors, 4054 slices

Preprocessing Complete:
- Training data: 289 tomograms, 363 motors, 3262 slices
- Validation data: 73 tomograms, 88 motors, 792 slices
- Dataset directory: kaggle/working/yolo_dataset
- YAML configuration: kaggle/working/yolo_dataset/dataset.yaml

Ready for YOLO training!


### Training and Testing

In [1]:
import wandb
wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: carlofinnegan (traintest) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [5]:
import os
import numpy as np
from PIL import Image
import shutil
from pathlib import Path
from tqdm import tqdm

def create_augmentations(dataset_root):
    """
    Apply 8 different transformations to images in a YOLO dataset and update labels accordingly.
    
    Transformations:
    1. Vertical flip
    2. 90° rotation
    3. Vertical flip + 90° rotation
    4. 180° rotation
    5. Vertical flip + 180° rotation
    6. 270° rotation
    7. Vertical flip + 270° rotation
    8. No transformation (original)
    """
    # Define directories
    images_dir = os.path.join(dataset_root, "images", "train")
    labels_dir = os.path.join(dataset_root, "labels", "train")
    
    # Create augmented directories
    augmented_images_dir = os.path.join(dataset_root, "images", "train_augmented")
    augmented_labels_dir = os.path.join(dataset_root, "labels", "train_augmented")
    
    os.makedirs(augmented_images_dir, exist_ok=True)
    os.makedirs(augmented_labels_dir, exist_ok=True)
    
    # Get all image files
    image_files = []
    for ext in ['.jpg', '.jpeg', '.png', '.tif', '.tiff', '.bmp']:
        image_files.extend(list(Path(images_dir).glob(f'*{ext}')))
        image_files.extend(list(Path(images_dir).glob(f'*{ext.upper()}')))
    
    print(f"Found {len(image_files)} images in training set.")
    
    # Process each image and its corresponding label
    for img_path in tqdm(image_files, desc="Processing images"):
        # Get base filename without extension
        base_name = os.path.splitext(os.path.basename(img_path))[0]
        ext = os.path.splitext(img_path)[1]
        
        # Find corresponding label file
        label_path = os.path.join(labels_dir, f"{base_name}.txt")
        if not os.path.exists(label_path):
            print(f"Warning: No label file found for {img_path}")
            continue
        
        # Read label file
        with open(label_path, 'r') as f:
            label_lines = f.readlines()
        
        # Load image
        try:
            img = Image.open(img_path)
            width, height = img.size
            
            # Process transformations
            transformations = [
                ("original", lambda i: i, lambda x, y, w, h: (x, y, w, h)),
                ("vflip", lambda i: i.transpose(Image.FLIP_TOP_BOTTOM), 
                 lambda x, y, w, h: (x, 1.0-y, w, h)),
                ("rot90", lambda i: i.transpose(Image.ROTATE_90), 
                 lambda x, y, w, h: (y, 1.0-x, h, w)),
                ("vflip_rot90", lambda i: i.transpose(Image.FLIP_TOP_BOTTOM).transpose(Image.ROTATE_90), 
                 lambda x, y, w, h: (1.0-y, 1.0-x, h, w)),
                ("rot180", lambda i: i.transpose(Image.ROTATE_180), 
                 lambda x, y, w, h: (1.0-x, 1.0-y, w, h)),
                ("vflip_rot180", lambda i: i.transpose(Image.FLIP_TOP_BOTTOM).transpose(Image.ROTATE_180), 
                 lambda x, y, w, h: (1.0-x, y, w, h)),
                ("rot270", lambda i: i.transpose(Image.ROTATE_270), 
                 lambda x, y, w, h: (1.0-y, x, h, w)),
                ("vflip_rot270", lambda i: i.transpose(Image.FLIP_TOP_BOTTOM).transpose(Image.ROTATE_270), 
                 lambda x, y, w, h: (y, x, h, w))
            ]
            
            for name, img_transform, bbox_transform in transformations:
                # Transform and save image
                transformed_img = img_transform(img)
                transformed_img.save(os.path.join(augmented_images_dir, f"{base_name}_{name}{ext}"))
                
                # Transform and save label
                transformed_labels = []
                for line in label_lines:
                    parts = line.strip().split()
                    if len(parts) >= 5:  # class x y w h format
                        class_id = parts[0]
                        x, y, w, h = map(float, parts[1:5])
                        
                        # For rotations that swap width and height
                        if "rot90" in name or "rot270" in name:
                            new_x, new_y, new_w, new_h = bbox_transform(x, y, w, h)
                        else:
                            new_x, new_y, new_w, new_h = bbox_transform(x, y, w, h)
                        
                        transformed_labels.append(f"{class_id} {new_x:.6f} {new_y:.6f} {new_w:.6f} {new_h:.6f}")
                    else:
                        transformed_labels.append(line.strip())  # Keep unchanged if not in standard format
                
                # Write transformed label file
                with open(os.path.join(augmented_labels_dir, f"{base_name}_{name}.txt"), 'w') as f:
                    f.write('\n'.join(transformed_labels))
                    
        except Exception as e:
            print(f"Error processing {img_path}: {e}")
    
    print("Augmentation complete!")
    print(f"Augmented images saved to: {augmented_images_dir}")
    print(f"Augmented labels saved to: {augmented_labels_dir}")

if __name__ == "__main__":
    # Replace with your actual dataset root directory
    dataset_root = "kaggle/working/yolo_dataset"
    create_augmentations(dataset_root)

Found 3262 images in training set.


Processing images: 100%|██████████| 3262/3262 [01:11<00:00, 45.89it/s]

Augmentation complete!
Augmented images saved to: kaggle/working/yolo_dataset/images/train_augmented
Augmented labels saved to: kaggle/working/yolo_dataset/labels/train_augmented


In [2]:
import os
import torch
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
from ultralytics import YOLO
import yaml
import pandas as pd
import json
import wandb
from wandb.integration.ultralytics import add_wandb_callback



wandb.init(project="BYU_Comp", job_type="training")



# Set random seeds for reproducibility
np.random.seed(42)
random.seed(42)
torch.manual_seed(42)

# Define paths for Kaggle environment
yolo_dataset_dir = "kaggle/working/yolo_dataset"
yolo_weights_dir = "kaggle/working/yolo_weights"
yolo_pretrained_weights = "yolov8n.pt"  # Path to pre-downloaded weights

# Create weights directory if it doesn't exist
os.makedirs(yolo_weights_dir, exist_ok=True)



def fix_yaml_paths(yaml_path):
    """
    Fix the paths in the YAML file to match the actual Kaggle directories
    
    Args:
        yaml_path (str): Path to the original dataset YAML file
        
    Returns:
        str: Path to the fixed YAML file
    """
    print(f"Fixing YAML paths in {yaml_path}")
    
    # Read the original YAML
    with open(yaml_path, 'r') as f:
        yaml_data = yaml.safe_load(f)
    
    # Update paths to use actual dataset location
    if 'path' in yaml_data:
        yaml_data['path'] = yolo_dataset_dir
    
    # Create a new fixed YAML in the working directory
    fixed_yaml_path = "kaggle/working/fixed_dataset.yaml"
    with open(fixed_yaml_path, 'w') as f:
        yaml.dump(yaml_data, f)
    
    print(f"Created fixed YAML at {fixed_yaml_path} with path: {yaml_data.get('path')}")
    return fixed_yaml_path



def plot_dfl_loss_curve(run_dir):
    """
    Plot the DFL loss curves for train and validation, marking the best model
    
    Args:
        run_dir (str): Directory where the training results are stored
    """
    # Path to the results CSV file
    results_csv = os.path.join(run_dir, 'results.csv')
    
    if not os.path.exists(results_csv):
        print(f"Results file not found at {results_csv}")
        return
    
    # Read results CSV
    results_df = pd.read_csv(results_csv)
    
    # Check if DFL loss columns exist
    train_dfl_col = [col for col in results_df.columns if 'train/dfl_loss' in col]
    val_dfl_col = [col for col in results_df.columns if 'val/dfl_loss' in col]
    
    if not train_dfl_col or not val_dfl_col:
        print("DFL loss columns not found in results CSV")
        print(f"Available columns: {results_df.columns.tolist()}")
        return
    
    train_dfl_col = train_dfl_col[0]
    val_dfl_col = val_dfl_col[0]
    
    # Find the epoch with the best validation loss
    best_epoch = results_df[val_dfl_col].idxmin()
    best_val_loss = results_df.loc[best_epoch, val_dfl_col]
    
    # Create the plot
    plt.figure(figsize=(10, 6))
    
    # Plot training and validation losses
    plt.plot(results_df['epoch'], results_df[train_dfl_col], label='Train DFL Loss')
    plt.plot(results_df['epoch'], results_df[val_dfl_col], label='Validation DFL Loss')
    
    # Mark the best model with a vertical line
    plt.axvline(x=results_df.loc[best_epoch, 'epoch'], color='r', linestyle='--', 
                label=f'Best Model (Epoch {int(results_df.loc[best_epoch, "epoch"])}, Val Loss: {best_val_loss:.4f})')
    
    # Add labels and legend
    plt.xlabel('Epoch')
    plt.ylabel('DFL Loss')
    plt.title('Training and Validation DFL Loss')
    plt.legend()
    plt.grid(True, linestyle='--', alpha=0.7)
    
    # Save the plot in the same directory as weights
    plot_path = os.path.join(run_dir, 'dfl_loss_curve.png')
    plt.savefig(plot_path)
    
    # Also save it to the working directory for easier access
    plt.savefig(os.path.join('kaggle/working', 'dfl_loss_curve.png'))
    
    print(f"Loss curve saved to {plot_path}")
    plt.close()
    
    # Return the best epoch info
    return best_epoch, best_val_loss



def train_yolo_model(yaml_path, pretrained_weights_path, epochs=30, batch_size=12, img_size=640):
    """
    Train a YOLO model on the prepared dataset
    
    Args:
        yaml_path (str): Path to the dataset YAML file
        pretrained_weights_path (str): Path to pre-downloaded weights file
        epochs (int): Number of training epochs
        batch_size (int): Batch size for training
        img_size (int): Image size for training
    """
    print(f"Loading pre-trained weights from: {pretrained_weights_path}")
    
    # Load a pre-trained YOLOv8 model
    model = YOLO(pretrained_weights_path)
    
    # Train the model with early stopping
    results = model.train(
        data=yaml_path,
        epochs=epochs,
        batch=batch_size,
        imgsz=img_size,
        project="yolo_weights_dir",
        name='motor_detector',
        exist_ok=True,
        patience=20,              # Early stopping if no improvement for 5 epochs
        save_period=5,           # Save checkpoints every 5 epochs
        val=True,                # Ensure validation is performed
        verbose=True
                                  # Show detailed output during training
    )
    
    # Get the path to the run directory
    run_dir = os.path.join(yolo_weights_dir, 'motor_detector')
    
    # Plot and save the loss curve
    best_epoch_info = plot_dfl_loss_curve(run_dir)
    
    if best_epoch_info:
        best_epoch, best_val_loss = best_epoch_info
        print(f"\nBest model found at epoch {best_epoch} with validation DFL loss: {best_val_loss:.4f}")
    
    return model, results



def predict_on_samples(model, num_samples=4):
    """
    Run predictions on random validation samples and display results
    
    Args:
        model: Trained YOLO model
        num_samples (int): Number of random samples to test
    """
    # Get validation images
    val_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
    if not os.path.exists(val_dir):
        print(f"Validation directory not found at {val_dir}")
        # Try train directory instead if val doesn't exist
        val_dir = os.path.join(yolo_dataset_dir, 'images', 'train')
        print(f"Using train directory for predictions instead: {val_dir}")
        
    if not os.path.exists(val_dir):
        print("No images directory found for predictions")
        return
    
    val_images = os.listdir(val_dir)
    
    if len(val_images) == 0:
        print("No images found for prediction")
        return
    
    # Select random samples
    num_samples = min(num_samples, len(val_images))
    samples = random.sample(val_images, num_samples)
    
    # Create figure
    fig, axes = plt.subplots(2, 2, figsize=(12, 12))
    axes = axes.flatten()
    
    for i, img_file in enumerate(samples):
        if i >= len(axes):
            break
            
        img_path = os.path.join(val_dir, img_file)
        
        # Run prediction
        results = model.predict(img_path, conf=0.25)[0]
        
        # Load and display the image
        img = Image.open(img_path)
        axes[i].imshow(np.array(img), cmap='gray')
        
        # Draw ground truth box if available (from filename)
        try:
            # This assumes your filenames contain coordinates in a specific format
            parts = img_file.split('_')
            y_part = [p for p in parts if p.startswith('y')]
            x_part = [p for p in parts if p.startswith('x')]
            
            if y_part and x_part:
                y_gt = int(y_part[0][1:])
                x_gt = int(x_part[0][1:].split('.')[0])
                
                box_size = 24
                rect_gt = Rectangle((x_gt - box_size//2, y_gt - box_size//2), 
                              box_size, box_size, 
                              linewidth=1, edgecolor='g', facecolor='none')
                axes[i].add_patch(rect_gt)
        except:
            pass  # Skip ground truth if parsing fails
        
        # Draw predicted boxes (red)
        if len(results.boxes) > 0:
            boxes = results.boxes.xyxy.cpu().numpy()
            confs = results.boxes.conf.cpu().numpy()
            
            for box, conf in zip(boxes, confs):
                x1, y1, x2, y2 = box
                rect_pred = Rectangle((x1, y1), x2-x1, y2-y1, 
                                     linewidth=1, edgecolor='r', facecolor='none')
                axes[i].add_patch(rect_pred)
                axes[i].text(x1, y1-5, f'{conf:.2f}', color='red')
        
        axes[i].set_title(f"Image: {img_file}\nGround Truth (green) vs Prediction (red)")
    
    plt.tight_layout()
    
    # Save the predictions plot
    plt.savefig(os.path.join('kaggle/working', 'predictions.png'))
    plt.show()


# Check and create a dataset YAML if needed
def prepare_dataset():
    """
    Check if dataset exists and create a proper YAML if needed
    
    Returns:
        str: Path to the YAML file to use for training
    """
    # Check if images exist
    train_images_dir = os.path.join(yolo_dataset_dir, 'images', 'train_augmented')
    val_images_dir = os.path.join(yolo_dataset_dir, 'images', 'val')
    train_labels_dir = os.path.join(yolo_dataset_dir, 'labels', 'train_augmented')
    val_labels_dir = os.path.join(yolo_dataset_dir, 'labels', 'val')
    
    # Print directory existence status
    print(f"Directory status:")
    print(f"- Train images dir exists: {os.path.exists(train_images_dir)}")
    print(f"- Val images dir exists: {os.path.exists(val_images_dir)}")
    print(f"- Train labels dir exists: {os.path.exists(train_labels_dir)}")
    print(f"- Val labels dir exists: {os.path.exists(val_labels_dir)}")
    
    # Check for original YAML file
    original_yaml_path = os.path.join(yolo_dataset_dir, 'dataset.yaml')
    
    if os.path.exists(original_yaml_path):
        print(f"Found original dataset.yaml at {original_yaml_path}")
        # Fix the paths in the YAML
        return fix_yaml_paths(original_yaml_path)
    else:
        print(f"Original dataset.yaml not found, creating a new one")
        
        # Create a new YAML file
        yaml_data = {
            'path': yolo_dataset_dir,
            'train': 'images/train_augmented',
            'val': 'images/train_augmented' if not os.path.exists(val_images_dir) else 'images/val',
            'names': {0: 'motor'}
        }
        
        new_yaml_path = "kaggle/working/dataset.yaml"
        with open(new_yaml_path, 'w') as f:
            yaml.dump(yaml_data, f)
            
        print(f"Created new YAML at {new_yaml_path}")
        return new_yaml_path



# Main execution
def main():
    print("Starting YOLO training process...")
    
    # Prepare dataset and get YAML path
    yaml_path = prepare_dataset()
    print(f"Using YAML file: {yaml_path}")
    
    # Print YAML file contents
    with open(yaml_path, 'r') as f:
        yaml_content = f.read()
    print(f"YAML file contents:\n{yaml_content}")
    
    # Train model
    print("\nStarting YOLO training...")
    model, results = train_yolo_model(
        yaml_path,
        pretrained_weights_path=yolo_pretrained_weights,
        epochs=30  # Using 30 epochs instead of 100 for faster training
    )
    
    print("\nTraining complete!")
    
    # Run predictions
    print("\nRunning predictions on sample images...")
    predict_on_samples(model, num_samples=4)

if __name__ == "__main__":
    main()

wandb.finish()

Starting YOLO training process...
Directory status:
- Train images dir exists: True
- Val images dir exists: True
- Train labels dir exists: True
- Val labels dir exists: True
Original dataset.yaml not found, creating a new one
Created new YAML at kaggle/working/dataset.yaml
Using YAML file: kaggle/working/dataset.yaml
YAML file contents:
names:
  0: motor
path: kaggle/working/yolo_dataset
train: images/train_augmented
val: images/val


Starting YOLO training...
Loading pre-trained weights from: yolov8n.pt
New https://pypi.org/project/ultralytics/8.3.89 available 😃 Update with 'pip install -U ultralytics'
Ultralytics 8.3.88 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3070, 7971MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=kaggle/working/dataset.yaml, epochs=30, time=None, patience=20, batch=12, imgsz=640, save=True, save_period=5, cache=False, device=None, workers=8, project=yolo_weights_dir, name=motor_detector, exist_ok=True, pretrained=True, 

train: Scanning /home/carlo/Documents/Github/W&B/kaggle/working/yolo_dataset/labels/train_augmented.cache... 26096 images, 0 backgrounds, 0 corrupt: 100%|██████████| 26096/26096 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))


val: Scanning /home/carlo/Documents/Github/W&B/kaggle/working/yolo_dataset/labels/val.cache... 792 images, 0 backgrounds, 0 corrupt: 100%|██████████| 792/792 [00:00<?, ?it/s]


Plotting labels to yolo_weights_dir/motor_detector/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: SGD(lr=0.01, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.00046875), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to yolo_weights_dir/motor_detector
Starting training for 30 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 33/33 [00:02<00:00, 15.07it/s]


                   all        792        792      0.857      0.847      0.905       0.44
Speed: 0.1ms preprocess, 1.2ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to yolo_weights_dir/motor_detector


lr/pg0,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg1,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
lr/pg2,▃▆██▇▇▇▇▆▆▆▆▅▅▅▅▄▄▄▄▃▃▃▃▂▂▂▂▁▁
metrics/mAP50(B),▂▄▁▆▆▄▅▅▆█▇▇▇▇█▇▇▇▇▇█▇████████
metrics/mAP50-95(B),▂▄▁▅▅▃▃▄▅█▇▆▇▆▇█▇▇▇▇▇▇▇██▇▇▇▇█
metrics/precision(B),▁▅▂▆▆▄▆▆▆██▆█▇▇▇▇▇▇█▇▇▇▇██▇▇▇█
metrics/recall(B),▃▄▁▆▅▅▅▅▇█▇▇▇▇█▇█▇█▇██████████
model/GFLOPs,▁
model/parameters,▁
model/speed_PyTorch(ms),▁
train/box_loss,█▆▅▅▅▄▄▄▄▃▃▃▃▃▃▃▃▃▂▂▂▂▂▂▁▁▁▁▁▁


Loss curve saved to kaggle/working/yolo_weights/motor_detector/dfl_loss_curve.png

Best model found at epoch 12 with validation DFL loss: 0.9020

Training complete!

Running predictions on sample images...

image 1/1 /home/carlo/Documents/Github/W&B/kaggle/working/yolo_dataset/images/val/tomo_6f83d4_z0120_y0424_x0293.jpg: 640x640 1 motor, 2.3ms
Speed: 1.4ms preprocess, 2.3ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/carlo/Documents/Github/W&B/kaggle/working/yolo_dataset/images/val/tomo_a37a5c_z0165_y0748_x0709.jpg: 640x640 1 motor, 2.8ms
Speed: 2.2ms preprocess, 2.8ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/carlo/Documents/Github/W&B/kaggle/working/yolo_dataset/images/val/tomo_37dd38_z0187_y0303_x0593.jpg: 640x640 (no detections), 2.9ms
Speed: 1.7ms preprocess, 2.9ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

image 1/1 /home/carlo/Documents/Github/W&B/kaggle/working/yolo_dataset/ima

<Figure size 1200x1200 with 4 Axes>